# Tensorflow Estimator

In [10]:
# Basic Imports 
import tensorflow as tf

import pandas as pd

import numpy as np 

from sklearn.model_selection import train_test_split


wines_df = pd.read_csv("../data/winequality.csv")



# tf doesn't like spaces in col names so I replace them with _ 

new_col_list = []
for col_name in wines_df.columns:
    new_col_names = col_name.replace(" ", "_")
    new_col_list.append(new_col_names)
wines_df.columns = new_col_list


# Change the type of the index values from [0....1598] to [wine_1....wine_1599]

wines_df.index += 1 # add 1 to index values to start wine specification from wine_1 rather than wine_0

index_as_string = wines_df.index.astype('str')

wines_df.index = 'wine_' + index_as_string


## Splitting the dataset to training, validation and test sets

In [2]:
# shuffle the data of the wines_df

wines_df = wines_df.sample(frac=1) 

# almost 70% training , 15% validation, 15% test set

intermediate_set, valid_set = train_test_split(wines_df, test_size=0.15) 
train_set, test_set = train_test_split(intermediate_set, test_size=0.15)

In [3]:
train_y = train_set.pop('quality')

In [4]:
valid_y = valid_set.pop('quality')

In [5]:
test_y = test_set.pop('quality')

In [ ]:
# sad
train_set has 1155 examples
for batch_size equal to 33 we have batch_count = 1155/33 = 35




1155/33

train_set.shape

valid_set.shape

test_set.shape

valid_set has 240 examples

for batch_size = 20 we have 240/ 20 = 12 number of batches

test_set has 204 examples
for batch_size = 17 we have 204/17 = 12 number of batches

In [14]:
train_batch_size = 33
valid_batch_size = 20
test_batch_size = 17

## Input functions

In [ ]:
# When to use df.copy


# This will change the original df because of df[0:1]
df = pd.DataFrame({'x': [1,2,3]})
df_sub = df[0:2]
display(df, df_sub)
df_sub.x = -10
print("The original df changed as well")
display(df)
print("As we changed dfsub")
display(df_sub)


# Instead If I create a copy I wont have this problem

print("We create the dataframes again ")

df = pd.DataFrame({'x': [1,2,3]})
df.sub = df[0:2].copy()
display(df, df_sub)
df_sub.x = -10
print("Will the original df change now?")
display(df)
print("Comparing it to dfsub hoping they are different")
display(df_sub)

In [157]:
# creating the input function for training, validation, prediction
def input_fn(dataframe, labels, batch_size):
    
    '''
    dataframe = train_set or valid_ set or test_set
    
    labels = train_y or valid_set or test_set
    
    batch_size = 33 for training or 20 for validation or 17 for test_set   
    '''
    
    # normalization_1
    df = dataframe.copy()
    min_values = df.min(axis=0)
    max_values = df.max(axis=0)
    max_minus_min = max_values - min_values
    df = df - min_values
    df = df.div(max_minus_min)

    
    
    
    
    
    dataset = tf.data.Dataset.from_tensor_slices((df.values, labels.values))
    dataset = dataset.batch(batch_size)
    
    
    return dataset



In [ ]:
data = input_fn(dataframe=valid_set, labels=valid_y, batch_size=valid_batch_size)

In [ ]:
# Doulepse
for row in data:

    feat , labels = row
    
    # build model graph 
    inputs = tf.keras.Input(shape=(11,))
    dense_1 = tf.keras.layers.Dense(units=20, input_shape=(33,11), activation='relu', dtype='float32')(inputs)
    dense_2 = tf.keras.layers.Dense(units=10, input_shape=(33,20), activation="relu", dtype='float32')(dense_1)
    model = tf.keras.Model(inputs=inputs, outputs=dense_2)
    
    

    model.summary()
    
    # model's output is the logits for every batch
    
    logit = model(feat)
    probs = tf.keras.activations.softmax(logit)
    class_predictions = tf.argmax(probs, axis=1, output_type=tf.int64)


    # check the first data point
    print(f"The logits of each class in the first data point are \n {logit[0,:]}")   
    print("\n")
    print(f"The probabilities of each class in the first data point are \n {probs[0,:]}")
    print("\n")
    print(f"The index of the most probable class for the first data point is \n {class_predictions[0]}")
    print("\n")




    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, tf.cast(probs, tf.float32)))
    # var_list = [dense_1.trainable_weights, dense_2.trainable_weights]
    
    
    print("The loss is {}".format(loss))

    break

## Model Function

In [ ]:
# model_fn without feature_columns

def model_fn(features, labels, mode, params):
    
    
    
    # building the model graph
    inputs = tf.keras.Input(shape=(11,))
    dense_1 = tf.keras.layers.Dense(20, activation='relu', name='dense_1', dtype=tf.float32)(inputs)
    dense_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2', dtype=tf.float32)(dense_1)
    model = tf.keras.Model(inputs=inputs, outputs=dense_2)
    
    
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        print(f"The model's variables are: {model.trainable_variables}")
    
        logits = model(features)
        probs = tf.keras.activations.softmax(logits)
        class_index = tf.argmax(probs, axis=1 , output_type=tf.int64)
    
   
        def my_acc(labels,probs):
            acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='my_acc')
            acc_metric.update_state(y_true=labels, y_pred=probs)
            return {'acc': acc_metric}
    
    
        print("\n","Hello user")
        print("\n")
        print(f"The logits of each class in the first data point are \n {logits[0,:]}")   
        print("\n")
        print(f"The probabilities of each class in the first data point are \n {probs[0,:]}")
        print("\n")
        print(f"The index of the most probable class for the first data point is \n {class_index[0]}")
        print("\n")
        print(f"The model's variables are: {model.trainable_variables}")
        
        @tf.function
        def loss():
            return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels,probs))
    

        if mode == tf.estimator.ModeKeys.TRAIN:

            
            opt = tf.keras.optimizers.Adam()
            #train_op = opt.minimize(loss,model.trainable_variables) doesn't work
            gradients = tape.gradient(loss(), model.trainable_variables)
            train_op = opt.apply_gradients(zip(gradients, model.trainable_variables))
            return tf.estimator.EstimatorSpec(mode, loss=loss(), train_op = train_op)
        
        
        
        
#          opt = tf.keras.optimizers.Adam()
#          with tf.GradientTape() as tape:
#              dl_dw = tape.gradient(loss(), var_list)
#              train_op = opt.apply_gradients(zip(dl_dw, var_list))
        
#         return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

        elif mode == tf.estimator.ModeKeys.EVAL:
            #sca = tf.keras.metrics.SparseCategoricalAccuracy()
            #update_op = sca.update_state(y_true=labels, y_pred=probs)
            #update_op = tf.keras.metrics.SparseCategoricalAccuracy.update_state(labels,probs)
            metrics = my_acc(labels,probs)
            return tf.estimator.EstimatorSpec(mode,loss=loss(), eval_metric_ops=metrics)








        elif mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'top_class_index' : tf.argmax(probs),
                'probs_of_classes' : probs,
                'logits_of_classes' : logits
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)




In [ ]:
my_estimator = tf.estimator.Estimator(model_fn = model_fn)

In [ ]:
my_estimator.train(input_fn=lambda:input_fn(train_set, train_y, train_batch_size), steps=2000)

In [ ]:
my_estimator.evaluate(input_fn=lambda:input_fn(valid_set, valid_y, valid_batch_size))

In [ ]:
preds = my_estimator.predict(input_fn=lambda:input_fn(test_set, test_y, test_batch_size), yield_single_examples=False)

In [ ]:
next(preds)

In [ ]:
next(preds)

In [ ]:
next(preds)

## Examples of function minimization

In [ ]:
# Example of minimizing a function f(x) = x^2-10x+25 with .minimize


x = tf.Variable(0,dtype=tf.float32, trainable=True)


@tf.function
def f_x():
    return x**2 -10*x + 25



for _ in range(200):
    print([x.numpy(), f_x().numpy()])
    opt = tf.keras.optimizers.SGD(0.01).minimize(f_x,[x])


    
    
    




In [ ]:
# Example of minimizing a function f(x) = x^2-10x+25 with GradientTape


x = tf.Variable([0], dtype=tf.float32, trainable=True)


@tf.function
def f_x():
    return x**2 -10*x +25

opt = tf.keras.optimizers.SGD(0.01)
for _ in range(200):
    print([x[0].numpy(), f_x().numpy()])
    with tf.GradientTape() as tape:
        df_dx = tape.gradient(f_x(),x)
        opt.apply_gradients(zip([df_dx],[x]))
    



In [ ]:
y_true = tf.constant([0,1,1])

In [ ]:
y_pred = tf.constant([[4, 2, 0],[1,5,3],[3,1,1]]) 

In [ ]:
tf.keras.metrics.sparse_categorical_accuracy(y_true,y_pred).numpy()

## Feature Columns

In [183]:
# model_fn with feature_columns 

def my_model_fn(features, labels, mode, params):
    
    # extract those from params dict
    feature_names = ['fixed_acidity','volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide','density','pH','sulphates','alcohol']   
    
    feature_columns =[]
    feature_layer_inputs = {}  # dictionary that maps feature names to tf.keras.Input layers
    for names in feature_names:
        feature_columns.append(tf.feature_column.numeric_column(names))
        feature_layer_inputs[names] = tf.keras.Input(dtype=tf.dtypes.float32, shape=(,11), batch_size=33, name=names)
        
        
    
    
    
    # model's architecture
    inputs = tf.
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)(feature_layer_inputs)
    dense_1 = tf.keras.layers.Dense(20, activation='relu',name='dense_1',dtype=tf.float32)(feature_layer)
    dense_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2', dtype=tf.float32)(dense_1)
    model = tf.keras.Model(inputs=inputs, outputs=dense_2)
    
    
    
    
    
    
        
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        print(f"The model's variables are: {model.trainable_variables}")
        logits = model(features)
        probs = tf.keras.activations.softmax(logits)
        class_index = tf.argmax(probs, axis=1 , output_type=tf.int64)
    
   
        def my_acc(labels,probs):
            acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='my_acc')
            acc_metric.update_state(y_true=labels, y_pred=probs)
            return {'acc': acc_metric}
    
    
        print("\n","Hello user")
        print("\n")
        print(f"The logits of each class in the first data point are \n {logits[0,:]}")   
        print("\n")
        print(f"The probabilities of each class in the first data point are \n {probs[0,:]}")
        print("\n")
        print(f"The index of the most probable class for the first data point is \n {class_index[0]}")
        print("\n")
        print(f"The model's variables are: {model.trainable_variables}")
        
        @tf.function
        def loss():
            return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels,probs))
    

        if mode == tf.estimator.ModeKeys.TRAIN:

            
            opt = tf.keras.optimizers.Adam()
            #train_op = opt.minimize(loss,model.trainable_variables) doesn't work
            gradients = tape.gradient(loss(), model.trainable_variables)
            train_op = opt.apply_gradients(zip(gradients, model.trainable_variables))
            return tf.estimator.EstimatorSpec(mode, loss=loss(), train_op = train_op)
        
        
        elif mode == tf.estimator.ModeKeys.EVAL:
            #sca = tf.keras.metrics.SparseCategoricalAccuracy()
            #update_op = sca.update_state(y_true=labels, y_pred=probs)
            #update_op = tf.keras.metrics.SparseCategoricalAccuracy.update_state(labels,probs)
            metrics = my_acc(labels,probs)
            return tf.estimator.EstimatorSpec(mode,loss=loss(), eval_metric_ops=metrics)








        elif mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'top_class_index' : tf.argmax(probs),
                'probs_of_classes' : probs,
                'logits_of_classes' : logits
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

In [184]:
my_estimator = tf.estimator.Estimator(model_fn = my_model_fn)

my_estimator.train(input_fn=lambda:input_fn(train_set, train_y, train_batch_size), steps=2000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_zveboyc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(33, 11)


ValueError: Input tensors to a Model must come from `tf.keras.Input`. Received: Tensor("concat:0", shape=(33, 11), dtype=float32) (missing previous layer metadata).